In [231]:
pip install -U langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [233]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [310]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [237]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [272]:
#load openAI api key
os.environ['OPENAI_API_KEY'] = 'your_ownkey'

In [241]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [274]:
llm = OpenAI(temperature=0.9, max_tokens=500)
from langchain.document_loaders import UnstructuredURLLoader

In [276]:
from bs4 import BeautifulSoup

urls = [
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
]

data = []

for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    text = soup.get_text()
    data.append(text)

In [280]:
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Wrap your text data into Document objects
documents = [Document(page_content=text) for text in data]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

docs = text_splitter.split_documents(documents)

In [206]:
len(docs)

36

In [208]:
docs[0]

Document(page_content='Wall Street rises as Tesla soars on AI optimism')

In [282]:

# Create the embeddings of the chunks using openAIEmbeddings
embeddings = OpenAIEmbeddings()

# Pass the documents and embeddings inorder to create FAISS vector index
vectorindex_openai = FAISS.from_documents(docs, embeddings)

In [296]:
import pickle
import ssl

class VectorIndexOpenAI:
    def __init__(self, some_data, ssl_context=None):
        self.some_data = some_data
        self.ssl_context = ssl_context  # Non-picklable attribute

    def __getstate__(self):
        # Exclude the non-picklable attribute from the state
        state = self.__dict__.copy()
        if 'ssl_context' in state:
            del state['ssl_context']
        return state

    def __setstate__(self, state):
        # Restore the state and reinitialize non-picklable attributes
        self.__dict__.update(state)
        self.ssl_context = None  # Or reinitialize as needed

# Example usage
vectorindex_openai = VectorIndexOpenAI(some_data={"key": "value"}, ssl_context=ssl.create_default_context())

# Save to a file
file_path = "vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorindex_openai, f)

# Load from the file
with open(file_path, "rb") as f:
    loaded_vectorindex_openai = pickle.load(f)

In [318]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

# Example document and embeddings
documents = [Document(page_content="Sample document content for FAISS indexing")]
embeddings = OpenAIEmbeddings()
faiss_index = FAISS.from_documents(documents, embeddings)

# Initialize your OpenAI model
llm = OpenAI()  # Ensure to configure this with necessary API keys or parameters

# Create the RetrievalQAWithSourcesChain instance
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=faiss_index.as_retriever())


In [338]:
query = "what is the price of Tiago iCNG?"
# query = "what are the main features of punch iCNG?"

langchain.debug=True

chain({"question": query}, return_only_outputs=True)


[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "what is the price of Tiago iCNG?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Sample document content for FAISS indexing",
      "question": "what is the price of Tiago iCNG?"
    }
  ]
}
[llm/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\nSample document content for FAISS indexing\nQuestion: what is the price of Tiago iCNG?\nRelevant text, if any:"
  ]
}
[llm/end] [chain:RetrievalQAWithSour

ValueError: Document prompt requires documents to have metadata variables: ['source']. Received document with missing metadata: ['source'].